### Create graphs subfolder in OpenFaaS directory

In [1]:
!mkdir -p graphs

### Install python package dependencies

In [2]:
!pip install kaleido plotly==5.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 17.9 MB/s eta 0:00:0000:0100:01


### Import packages to graph results

In [3]:
from glob import glob
import pandas as pd
import json
#import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'iframe'
# import plotly.offline as py
# py.init_notebook_mode(connected=True)
# Used for debug
# from pprint import pprint

### Set default color palete and size for making plots

In [4]:
colorPallet = ["rgba(151, 209, 233, 255)", "rgba(0, 120, 179, 255)", "rgba(179, 223, 146, 255)", "rgba(49, 169, 90, 255)", "rgba(227, 136, 220, 255)", "rgba(127, 0, 255, 255)", "rgba(255, 128, 0, 255)"]
bgColor = 'rgba(245,245,255,255)'

baseFigureWidth = 700
baseFigureHeight = 350

### glob all the log files into a list, then create a dataframe with all json data

In [5]:
bwa_normal_logs = glob("../logs/openfaas/kops_multi-tenant_1x_8vcpu_16gb/bwa/*normal*")
normal_df_list = []
for log in bwa_normal_logs:
    with open(log) as f:
        obj = json.load(f)
    # cpuInfo is messing up the dataframe... remove for now
    obj.pop('cpuInfo')
    normal_df_list.append(pd.DataFrame([obj]))
normal_df = pd.concat(normal_df_list, ignore_index=True)

bwa_tumor_logs = glob("../logs/openfaas/kops_multi-tenant_1x_8vcpu_16gb/bwa/*tumor*")
tumor_df_list = []
for log in bwa_tumor_logs:
    with open(log) as f:
        obj = json.load(f)
    # cpuInfo is messing up the dataframe... remove for now
    obj.pop('cpuInfo')
    tumor_df_list.append(pd.DataFrame([obj]))
tumor_df = pd.concat(tumor_df_list, ignore_index=True)

# Make bar plots to profile bwa function data

In [6]:
fig = make_subplots(specs = [[{"secondary_y": False}]])

# Add figure title
fig.update_layout(
    barmode='stack',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.47
    ),
    margin=dict(
        t=0,
        b=1,
        l=1,
        r=1,
        autoexpand=True
    ),
    font=dict(
        size=16
    ),
    plot_bgcolor=bgColor
)

profileData = pd.DataFrame()
profileData['payload'] = ["Normal", "Tumor"]
profileData['cpuUser'] = [normal_df['cpuUserDelta'].mean(), tumor_df['cpuUserDelta'].mean()]
profileData['cpuIdle'] = [normal_df['cpuIdleDelta'].mean(), tumor_df['cpuIdleDelta'].mean()]
profileData['cpuKernel'] = [normal_df['cpuKernelDelta'].mean(), tumor_df['cpuKernelDelta'].mean()]
profileData['runtime'] = [round(normal_df['runtime'].mean() / 1000), round(tumor_df['runtime'].mean() / 1000)]
profileData['runtimeText'] = [f"Runtime:<br>{value} s" for value in profileData['runtime']]

fig.add_trace(go.Bar(x = profileData["payload"],
                     y = profileData["cpuKernel"] / 100,
                     name = "CPU Kernel", marker_color=colorPallet[2]),
              secondary_y=False)

fig.add_trace(go.Bar(x = profileData["payload"],
                     y = profileData["cpuUser"] / 100,
                     name = "CPU User", marker_color=colorPallet[1]),
              secondary_y=False)

fig.add_trace(go.Bar(x = profileData["payload"],
                     y = profileData["cpuIdle"] / 100,
                     name = "CPU Idle", marker_color=colorPallet[0],
                     text=profileData['runtimeText'], textposition="outside"),
              secondary_y=False)

# Set x-axis title
fig.update_xaxes(title_text="Payloads")

# Set y-axes titles
fig.update_yaxes(title_text="CPU Time (s)", range=[0, 12000], secondary_y=False)
fig.update_yaxes(title_text="Function Runtime (ms)", secondary_y=True)

# py.iplot(fig)

fig.show()
# Save graph
# fig.write_image("graphs/bwa_profile_bars.pdf", width=baseFigureWidth, engine="kaleido", height=baseFigureHeight)